In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
df = pd.read_csv("../files/blocks_0.csv")
inputs = pd.read_csv("inputs.csv")
outputs = pd.read_csv("outputs.csv")

In [3]:
inputs = inputs.drop("Unnamed: 0", axis=1).drop_duplicates(ignore_index=True)
outputs = outputs.drop("Unnamed: 0", axis=1).drop_duplicates(ignore_index=True)

In [6]:
inputs.head(100)

,spent_transaction_hash,spent_output_index,sequence,type,addresses,values,hash
0,4d05f565163f69ccab77b9f548a90ec192b53c32bc7e14...,0,4294967295,scripthash,3FLVi4tqMBedc9LUwhJDZzKewDy7T5y37y,300000.0,d00a570dfb048df63163ace688c64149d9de8dc5a02754...
1,61932b239e3741d1ab408346dc77c69916ba617c6cdf0e...,0,4294967294,scripthash,35cc4bFixz8BVbXdrDKS7opaX5nCGsRv4B,1813533.0,50e9acd87b72ae5e0d11f351e1eed4ea5c4dbfacd43afb...
2,a13bec5b88ef97bd49ae2e189c7c3edae329d84c9a54e1...,0,4294967294,scripthash,32TyGcMXKjziJnuDyNBQNMuxAmPy5Gv5pp,1750000.0,50e9acd87b72ae5e0d11f351e1eed4ea5c4dbfacd43afb...
3,fc144488ec2607ab23985ea84044e002a711333b0e22f4...,33,4294967294,scripthash,3Dxi5FvKNN8DLSAtL88FYki7wStUVCeFBZ,305291.0,50e9acd87b72ae5e0d11f351e1eed4ea5c4dbfacd43afb...
4,1eb2e4822e656a8cec743d8873704eababcc9638fcff83...,46,4294967295,pubkeyhash,1PA3eogBCv3n1debNLJerJzTrPRKjscJfd,98757.0,37bf662918fa6d9796754d85065426075d8fe7dbc52e48...
...,...,...,...,...,...,...,...
95,3e98628992d8fea1c610149e17956e93777980472ffa65...,1,4294967294,scripthash,3QykhQregtNodXQYiQS3etCrDzy7hJ14mr,329663.0,f37bc9f455538f57b3f4c30a31bc637a6968e0e4b0222d...
96,a51ea80f8d5791579998032266c7a7ea7d2e8c32ef268e...,0,4294967294,scripthash,32Enoqg7376BUcBQoKRwp12CoRzqafoaAU,344834.0,f37bc9f455538f57b3f4c30a31bc637a6968e0e4b0222d...
97,806193a7a7adfcb2db8dc128ccfa3f1e1b5bacc14e9b45...,1,4294967294,scripthash,3NgwTTadEx7bQVEnGWS11Cghgm2DxpkfCt,105056.0,f37bc9f455538f57b3f4c30a31bc637a6968e0e4b0222d...
98,ed4179205664c26a6cf26518e70b80b55aa6269686ff03...,11,4294967294,scripthash,38Kc1euSQSaXwXT6BMbtzMKor9UX7es8DU,285889.0,f37bc9f455538f57b3f4c30a31bc637a6968e0e4b0222d...


In [1]:
#!pip install gephistreamer

In [16]:
from gephistreamer import graph
from gephistreamer import streamer

def build_mask(df, values, key):
    masks = [df[key] == v for v in values]
    try:
        mask = np.zeros(masks[0].shape[0])
    except IndexError: # we dont have this address in the input table
        return None, None
    for m in masks:
        mask = mask | m

    return masks, mask


def make_dot(address):
    
    #node_attr = dict(style="filled", shape="circle", fontsize="12", ranksep="0.1", height="0.2")
    stream = streamer.Streamer(streamer.GephiWS(hostname="localhost", port=8080, workspace="workspace1"))
    seen = set()
    scale = 1e9
    
    
    
    def add_nodes(addresses):
        
        _, input_mask = build_mask(inputs, addresses, "addresses")
    
        hash_values = inputs[input_mask]["hash"].values.tolist()
        all_masks, output_mask = build_mask(outputs, hash_values, "hash")
        if all_masks is None:
            return
        for address, mask in zip(addresses, all_masks):
            if address not in seen:
                stream.add_node(graph.Node(address[:5] + "..."))
                for edge, value in zip(outputs[mask]["addresses"].tolist(), outputs[mask]["values"].tolist()):
                    stream.add_node(graph.Node(edge[:5] + "...", size=value/scale))
                    stream.add_edge(graph.Edge(address[:5] + "...", edge[:5] + "...", label=value/scale))
                
                seen.add(address)

                print(f"{address} sent a transaction to the folowing adresses")
                print("\n".join(outputs[mask]["addresses"].tolist()))
                add_nodes(outputs[output_mask]["addresses"].values.tolist())
            
            
            return

            #for address in addresses:
            #    print(f"\t{address}")
            #display(inputs[input_mask])
            #display(outputs[output_mask])
    
    add_nodes(address)
    return

#""
#
make_dot(["3FLVi4tqMBedc9LUwhJDZzKewDy7T5y37y"]) # random address from bitcoinabuse
#g.view()

3FLVi4tqMBedc9LUwhJDZzKewDy7T5y37y sent a transaction to the folowing adresses
bc1qem9jxe40pdrawhfzmy22n4mfrj4ud3hl905mxk
bc1qylz7c38nplv4apkuwh36jfsaez5vlnv6hfx7jz
bc1qem9jxe40pdrawhfzmy22n4mfrj4ud3hl905mxk sent a transaction to the folowing adresses
bc1q0pqchldanqv2u4n6wtgrad2wmyejk4fjttg4ps
